In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import warnings
warnings.filterwarnings(action='ignore')

In [15]:
labels_dataframe=pd.read_csv('../input/paddy-disease-classification/train.csv')
sub_df=pd.read_csv('../input/paddy-disease-classification/sample_submission.csv')

In [16]:
labels_dataframe

In [17]:
sub_df

In [18]:
diseases_types=sorted(list(set(labels_dataframe['label'])))
n_classes=len(diseases_types)
print(n_classes)
print(diseases_types)

In [19]:
train_data= tf.keras.utils.image_dataset_from_directory("/kaggle/input/paddy-disease-classification/train_images/",
                                                        labels='inferred',
                                                        label_mode='categorical',
                                                        class_names=diseases_types,
                                                        color_mode='rgb',
                                                        image_size=(680,480),
                                                        shuffle=True,
                                                        validation_split=0.2,
                                                        subset='training',
                                                        seed=42)

val_data= tf.keras.utils.image_dataset_from_directory("/kaggle/input/paddy-disease-classification/train_images/",
                                                        labels='inferred',
                                                        label_mode='categorical',
                                                        class_names=diseases_types,   
                                                        color_mode='rgb',
                                                        image_size=(680,480),
                                                        shuffle=True,
                                                        validation_split=0.2,
                                                        subset='validation',
                                                        seed=42)

In [20]:
train_data

In [21]:
val_data

In [22]:
from tensorflow.keras import models,layers

In [23]:
resnet_model=models.Sequential()

pretrained_model=tf.keras.applications.ResNet50(include_top=False,input_shape=(680,480,3),pooling='avg',classes=10,weights='imagenet')
    
for layer in pretrained_model.layers:
    layer.trainable=False
    
    
resnet_model.add(pretrained_model)
resnet_model.add(layers.Flatten())
resnet_model.add(layers.Dense(512,activation='relu'))
resnet_model.add(layers.Dense(99,activation='relu'))
resnet_model.add(layers.Dense(10,activation='softmax'))

In [24]:
resnet_model.summary()

In [25]:
tf.keras.utils.plot_model(resnet_model)

In [26]:
resnet_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
resnet_model.fit(train_data,validation_data=val_data,epochs=20)

In [27]:
test_data=tf.keras.utils.image_dataset_from_directory("/kaggle/input/paddy-disease-classification/test_images/",
                                                        labels=None,
                                                        label_mode=None,
                                                        color_mode='rgb',
                                                        image_size=(680,480),
                                                        shuffle=False)

In [28]:
predict=tf.argmax(resnet_model.predict(test_data),axis=-1)
predict

In [29]:
predictions=[]
for i in predict:
    predictions.append(diseases_types[i])

In [30]:
predictions[:10]

In [31]:
sub_df=sub_df.drop(columns='label',axis=1)
sub_df

In [32]:
submission=pd.DataFrame({'image_id':sub_df['image_id'],'label':predictions})
submission

In [33]:
submission.to_csv('Paddy Doctor.csv',index=None)